In [1]:
#!/usr/bin/env python
import os
import csv
import time
import numpy as np
import tensorflow as tf
from tensorflow.contrib.rnn import BasicLSTMCell

import pickle
from typing import List

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


# Model Parameters

In [2]:
# from .decoding import get_words_from_chars
# from .config import Params, CONST
# from src.model import crnn_fn
# from src.data_handler import data_loader
# from src.data_handler import preprocess_image_for_prediction

# from src.config import Params, Alphabet, import_params_from_json

csv_files_train = "../data/train.csv"
csv_files_eval = "../data/valid.csv"
output_model_dir = "./estimator"
output_graph_dir = "./graph"
if not os.path.isdir(output_model_dir):
    os.mkdir(output_model_dir)
if not os.path.isdir(output_graph_dir):
    os.mkdir(output_graph_dir)


n_epochs = 10
gpu = "" # help="GPU 0,1 or '' ", default=''

# train_batch_size=64
# eval_batch_size=64
train_batch_size=10
eval_batch_size=10
learning_rate=1e-3  # 1e-3 recommended
learning_decay_rate=0.95
learning_decay_steps=5000
evaluate_every_epoch=5
save_interval=5e3
# input_shape=(129, 369) #iam shape
# input_shape=(117, 1669) # bentham shape1
input_shape=(181, 1833) # bentham shape2
optimizer='adam'
digits_only=False
alphabet=" !\"#&'()*+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXY[]_abcdefghijklmnopqrstuvwxyz|"
# alphabet="abcdefghijklmnopqrstuvwxyz"
alphabet_decoding='same'
alphabet_codes = list(range(len(alphabet)))
n_classes = len(alphabet)
csv_delimiter='\t'
keep_prob_dropout = 0.7


# Start of tensor graph

In [3]:
# needed for quickly making convolutional layers
def weightVar(shape, mean=0.0, stddev=0.02, name='weights'):
    init_w = tf.truncated_normal(shape=shape, mean=mean, stddev=stddev)
    return tf.Variable(init_w, name=name)


def biasVar(shape, value=0.0, name='bias'):
    init_b = tf.constant(value=value, shape=shape)
    return tf.Variable(init_b, name=name)


def conv2d(input, filter, strides=[1, 1, 1, 1], padding='SAME', name=None):
    return tf.nn.conv2d(input, filter, strides=strides, padding=padding, name=name)

In [4]:
#input_tensor = features['images']
input_tensor = tf.placeholder(tf.float32, [None, input_shape[0], input_shape[1], 1])
labels = tf.placeholder(tf.string, [None])
is_training = True

if input_tensor.shape[-1] == 1:
    input_channels = 1
elif input_tensor.shape[-1] == 3:
    input_channels = 3
else:
    raise NotImplementedError

# Deep cnn

In [5]:
with tf.variable_scope('deep_cnn'):
    # conv1 - maxPool2x2
    with tf.variable_scope('layer1'):
        W = weightVar([3, 3, input_channels, 64])
        b = biasVar([64])
        conv = conv2d(input_tensor, W, name='conv')
        out = tf.nn.bias_add(conv, b)
        conv1 = tf.nn.relu(out)
        pool1 = tf.nn.max_pool(conv1, [1, 2, 2, 1], strides=[1, 2, 2, 1],
                               padding='SAME', name='pool')

    # conv2 - maxPool 2x2
    with tf.variable_scope('layer2'):
        W = weightVar([3, 3, 64, 128])
        b = biasVar([128])
        conv = conv2d(pool1, W)
        out = tf.nn.bias_add(conv, b)
        conv2 = tf.nn.relu(out)
        pool2 = tf.nn.max_pool(conv2, [1, 2, 2, 1], strides=[1, 2, 2, 1],
                               padding='SAME', name='pool1')

    # conv3 - w/batch-norm (as source code, not paper)
    with tf.variable_scope('layer3'):
        W = weightVar([3, 3, 128, 256])
        b = biasVar([256])
        conv = conv2d(pool2, W)
        out = tf.nn.bias_add(conv, b)
        b_norm = tf.layers.batch_normalization(out, axis=-1,
                                               training=is_training, name='batch-norm')
        conv3 = tf.nn.relu(b_norm, name='ReLU')

    # conv4 - maxPool 2x1
    with tf.variable_scope('layer4'):
        W = weightVar([3, 3, 256, 256])
        b = biasVar([256])
        conv = conv2d(conv3, W)
        out = tf.nn.bias_add(conv, b)
        conv4 = tf.nn.relu(out)
        pool4 = tf.nn.max_pool(conv4, [1, 2, 2, 1], strides=[1, 2, 1, 1],
                               padding='SAME', name='pool4')

    # conv5 - w/batch-norm
    with tf.variable_scope('layer5'):
        W = weightVar([3, 3, 256, 512])
        b = biasVar([512])
        conv = conv2d(pool4, W)
        out = tf.nn.bias_add(conv, b)
        b_norm = tf.layers.batch_normalization(out, axis=-1,
                                               training=is_training, name='batch-norm')
        conv5 = tf.nn.relu(b_norm)

    # conv6 - maxPool 2x1 (as source code, not paper)
    with tf.variable_scope('layer6'):
        W = weightVar([3, 3, 512, 512])
        b = biasVar([512])
        conv = conv2d(conv5, W)
        out = tf.nn.bias_add(conv, b)
        conv6 = tf.nn.relu(out)
        pool6 = tf.nn.max_pool(conv6, [1, 2, 2, 1], strides=[1, 2, 1, 1],
                               padding='SAME', name='pool6')

    # conv 7 - w/batch-norm (as source code, not paper)
    with tf.variable_scope('layer7'):
        W = weightVar([2, 2, 512, 512])
        b = biasVar([512])
        conv = conv2d(pool6, W, padding='VALID')
        out = tf.nn.bias_add(conv, b)
        b_norm = tf.layers.batch_normalization(out, axis=-1,
                                               training=is_training, name='batch-norm')
        conv7 = tf.nn.relu(b_norm)

    # reshape output
    with tf.variable_scope('Reshaping_cnn'):
        shape = conv7.get_shape().as_list()  # [batch, height, width, features]
        shape_tens = tf.shape(conv7)
        transposed = tf.transpose(conv7, perm=[0, 2, 1, 3],
                                  name='transposed')  # [batch, width, height, features]
        conv_out = tf.reshape(transposed, [shape_tens[0], -1, shape[1] * shape[3]],
                                   name='reshaped')  # [batch, width, height x features]

# Bidirectional lstm

In [6]:
# logprob, raw_pred = deep_bidirectional_lstm(conv, params=parameters, summaries=False)

# def deep_bidirectional_lstm(inputs: tf.Tensor, params: Params, summaries: bool=True) -> tf.Tensor:
# Prepare data shape to match `bidirectional_rnn` function requirements
# Current data input shape: (batch_size, n_steps, n_input) "(batch, time, height)"

list_n_hidden = [256, 256]

with tf.name_scope('deep_bidirectional_lstm'):
    # Forward direction cells
    fw_cell_list = [BasicLSTMCell(nh, forget_bias=1.0) for nh in list_n_hidden]
    # Backward direction cells
    bw_cell_list = [BasicLSTMCell(nh, forget_bias=1.0) for nh in list_n_hidden]

    lstm_net, _, _ = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(fw_cell_list,
                                                                    bw_cell_list,
                                                                    conv_out, # THE INPUT
                                                                    dtype=tf.float32
                                                                    )

    # Dropout layer
    lstm_net = tf.nn.dropout(lstm_net, keep_prob=keep_prob_dropout)

    with tf.variable_scope('Reshaping_rnn'):
        shape = lstm_net.get_shape().as_list()  # [batch, width, 2*n_hidden]
        rnn_reshaped = tf.reshape(lstm_net, [-1, shape[-1]])  # [batch x width, 2*n_hidden]

    with tf.variable_scope('fully_connected'):
        W = weightVar([list_n_hidden[-1]*2, n_classes])
        b = biasVar([n_classes])
        fc_out = tf.nn.bias_add(tf.matmul(rnn_reshaped, W), b)

    shape_tens = tf.shape(lstm_net)
    lstm_out = tf.reshape(fc_out, [shape_tens[0], -1, n_classes], name='reshape_out')  # [batch, width, n_classes]

    raw_pred = tf.argmax(tf.nn.softmax(lstm_out), axis=2, name='raw_prediction')

    # Swap batch and time axis
    lstm_out = tf.transpose(lstm_out, [1, 0, 2], name='transpose_time_major')  # [width(time), batch, n_classes]
    

# CTC loss

In [7]:
# Set up for loss and training

# Compute seq_len from image width
n_pools = 2*2  # 2x2 pooling in dimension W on layer 1 and 2
seq_len_inputs = tf.divide([input_shape[1]]*train_batch_size, n_pools,
                           name='seq_len_input_op') - 1

predictions_dict = {'prob': lstm_out,
                    'raw_predictions': raw_pred,
                    }


# Get keys (letters) and values (integer stand ins for letters)
# Alphabet and codes
keys = [c for c in alphabet] # the letters themselves
values = alphabet_codes # integer representations


# Create non-string labels from the keys and values above
# Convert string label to code label
with tf.name_scope('str2code_conversion'):
    table_str2int = tf.contrib.lookup.HashTable(tf.contrib.lookup.KeyValueTensorInitializer(keys, values), -1)
    splited = tf.string_split(labels, delimiter='')  # TODO change string split to utf8 split in next tf version
    codes = table_str2int.lookup(splited.values)
    sparse_code_target = tf.SparseTensor(splited.indices, codes, splited.dense_shape)

seq_lengths_labels = tf.bincount(tf.cast(sparse_code_target.indices[:, 0], tf.int32),
                                 minlength=tf.shape(predictions_dict['prob'])[1])


# Use ctc loss on probabilities from lstm output
# Loss
# ----
# >>> Cannot have longer labels than predictions -> error
with tf.control_dependencies([tf.less_equal(sparse_code_target.dense_shape[1], tf.reduce_max(tf.cast(seq_len_inputs, tf.int64)))]):
    loss_ctc = tf.nn.ctc_loss(labels=sparse_code_target,
                              inputs=predictions_dict['prob'],
                              sequence_length=tf.cast(seq_len_inputs, tf.int32),
                              preprocess_collapse_repeated=False,
                              ctc_merge_repeated=True,
                              ignore_longer_outputs_than_inputs=True,  # returns zero gradient in case it happens -> ema loss = NaN
                              time_major=True)
    loss_ctc = tf.reduce_mean(loss_ctc)
    loss_ctc = tf.Print(loss_ctc, [loss_ctc], message='* Loss : ')

# CTC decoding

In [8]:
def get_words_from_chars(characters_list: List[str], sequence_lengths: List[int], name='chars_conversion'):
    with tf.name_scope(name=name):
        def join_charcaters_fn(coords):
            return tf.reduce_join(characters_list[coords[0]:coords[1]])

        def coords_several_sequences():
            end_coords = tf.cumsum(sequence_lengths)
            start_coords = tf.concat([[0], end_coords[:-1]], axis=0)
            coords = tf.stack([start_coords, end_coords], axis=1)
            coords = tf.cast(coords, dtype=tf.int32)
            return tf.map_fn(join_charcaters_fn, coords, dtype=tf.string)

        def coords_single_sequence():
            return tf.reduce_join(characters_list, keep_dims=True)

        words = tf.cond(tf.shape(sequence_lengths)[0] > 1,
                        true_fn=lambda: coords_several_sequences(),
                        false_fn=lambda: coords_single_sequence())

        return words

In [9]:
with tf.name_scope('code2str_conversion'):
    keys = tf.cast(alphabet_codes, tf.int64)
    values = [c for c in alphabet]

    table_int2str = tf.contrib.lookup.HashTable(tf.contrib.lookup.KeyValueTensorInitializer(keys, values), '?')

    sparse_code_pred, log_probability = tf.nn.ctc_beam_search_decoder(predictions_dict['prob'],
                                                                      sequence_length=tf.cast(seq_len_inputs, tf.int32),
                                                                      merge_repeated=False,
                                                                      beam_width=100,
                                                                      top_paths=2)
    # Score
    predictions_dict['score'] = tf.subtract(log_probability[:, 0], log_probability[:, 1])
    # around 10.0 -> seems pretty sure, less than 5.0 bit unsure, some errors/challenging images
    sparse_code_pred = sparse_code_pred[0]

    sequence_lengths_pred = tf.bincount(tf.cast(sparse_code_pred.indices[:, 0], tf.int32),
                                        minlength=tf.shape(predictions_dict['prob'])[1])

    pred_chars = table_int2str.lookup(sparse_code_pred)
    predictions_dict['words'] = get_words_from_chars(pred_chars.values, sequence_lengths=sequence_lengths_pred)

    tf.summary.text('predicted_words', predictions_dict['words'][:10])

In [10]:
with tf.name_scope('evaluation'):
    CER = tf.metrics.mean(tf.edit_distance(sparse_code_pred, tf.cast(sparse_code_target, dtype=tf.int64)), name='CER')
    CER = tf.reduce_mean(tf.edit_distance(sparse_code_pred, tf.cast(sparse_code_target, dtype=tf.int64)), name='CER')

    # Convert label codes to decoding alphabet to compare predicted and groundtrouth words
    target_chars = table_int2str.lookup(tf.cast(sparse_code_target, tf.int64))
    target_words = get_words_from_chars(target_chars.values, seq_lengths_labels)
    accuracy = tf.metrics.accuracy(target_words, predictions_dict['words'], name='accuracy')

    eval_metric_ops = {
                       'eval/accuracy': accuracy,
                       'eval/CER': CER,
                       }
    CER = tf.Print(CER, [CER], message='-- CER : ')
    accuracy = tf.Print(accuracy, [accuracy], message='-- Accuracy : ')


# Set up optimizer

In [11]:
# Create the learning rate as well as a moving average
global_step = tf.train.get_or_create_global_step()
# # Create an ExponentialMovingAverage object
ema = tf.train.ExponentialMovingAverage(decay=0.99, num_updates=global_step, zero_debias=True)
# Create the shadow variables, and add op to maintain moving averages
maintain_averages_op = ema.apply([loss_ctc])
loss_ema = ema.average(loss_ctc)

# Train op
# --------
learning_rate = tf.train.exponential_decay(learning_rate, global_step,
                                           learning_decay_steps, learning_decay_rate,
                                           staircase=True)

# Set up optimizer
if optimizer == 'ada':
    optimizer = tf.train.AdadeltaOptimizer(learning_rate)
elif optimizer == 'adam':
    optimizer = tf.train.AdamOptimizer(learning_rate, beta1=0.5)
elif optimizer == 'rms':
    optimizer = tf.train.RMSPropOptimizer(learning_rate)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
opt_op = optimizer.minimize(loss_ctc, global_step=global_step)
with tf.control_dependencies(update_ops + [opt_op]):
    train_op = tf.group(maintain_averages_op)

# Import data

In [12]:
# get the details to make all images the same size

with open(csv_files_train, "r") as f:
    f_lines = f.read().splitlines()[1:]
    filenames = [l.split("\t")[0] for l in f_lines]
    label_list = [l.split("\t")[1] for l in f_lines]

datasize = len(label_list)
filenames = tf.constant(filenames)
label_list = tf.constant(label_list)
    
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_png(image_string, channels=1)
    image_resized = tf.image.resize_image_with_crop_or_pad(image_decoded,
                                                           input_shape[0],
                                                           input_shape[1])
    return image_resized, label
    
dataset = tf.data.Dataset.from_tensor_slices((filenames, label_list))
dataset = dataset.map(_parse_function)
dataset = dataset.batch(train_batch_size)
iterator = dataset.make_initializable_iterator()
next_batch = iterator.get_next()

# Run training session

In [ ]:
saver = tf.train.Saver()

losses = []
cers = []
accs = []

with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(tf.tables_initializer())
#     saver.restore(sess, output_model_dir+"/model.ckpt")
    
    writer = tf.summary.FileWriter(output_graph_dir+'/crnn', sess.graph)
    n_batches = int(datasize / train_batch_size)
    for i in range(n_epochs):
        print("---------------------------------------------------------")
        print("Starting epoch", i)
        sess.run(iterator.initializer)
        for j in range(n_batches):
            input_tensor_b, labels_b = sess.run(next_batch)
            try:
                _, cer, acc, loss, preddict, scp, pc = sess.run([train_op, CER, accuracy, loss_ctc, predictions_dict,
                                                              sparse_code_pred, pred_chars],
                             feed_dict={input_tensor: input_tensor_b, labels: labels_b})
                losses.append(loss)
                cers.append(cer)
                accs.append(accs)
                pickle.dump([cers, accs, losses, preddict, scp, pc], open(output_model_dir+"/preds.pkl", "wb"))
                saver.save(sess, output_model_dir+"/model.ckpt")
                
                print('batch: {0}, loss: {3} \n\tCER: {1}, accuracy: {2}'.format(j, cer, acc, loss))
            except:
                print("Error at ", j)
        print('Avg Epoch time: {0} seconds'.format((time.time() - start_time)/(1.0*(i+1))))
            
    print('Total time: {0} seconds'.format(time.time() - start_time))
    print('Optimization Finished!') 

---------------------------------------------------------
Starting epoch 0
batch: 0, loss: 1747.891357421875 
	CER: 3.6099853515625, accuracy: [0. 0.]
batch: 1, loss: 1070.522216796875 
	CER: 4.66179084777832, accuracy: [0. 0.]
batch: 2, loss: 488.2562561035156 
	CER: 1.2632659673690796, accuracy: [0. 0.]
batch: 3, loss: 260.8626403808594 
	CER: 0.9572297930717468, accuracy: [0. 0.]
batch: 4, loss: 200.49440002441406 
	CER: 0.9642949104309082, accuracy: [0. 0.]
batch: 5, loss: 172.14146423339844 
	CER: 1.0589720010757446, accuracy: [0. 0.]
batch: 6, loss: 170.50469970703125 
	CER: 0.901185154914856, accuracy: [0. 0.]
batch: 7, loss: 208.1443634033203 
	CER: 0.8527774810791016, accuracy: [0. 0.]
batch: 8, loss: 189.603515625 
	CER: 0.7966344356536865, accuracy: [0. 0.]
batch: 9, loss: 190.32711791992188 
	CER: 1.4789849519729614, accuracy: [0. 0.]
batch: 10, loss: 150.1199188232422 
	CER: 0.8787711262702942, accuracy: [0. 0.]
batch: 11, loss: 184.90029907226562 
	CER: 0.860216498374939,

batch: 102, loss: 149.83120727539062 
	CER: 0.8207523226737976, accuracy: [0. 0.]
batch: 103, loss: 91.45783996582031 
	CER: 0.8748781085014343, accuracy: [0. 0.]
batch: 104, loss: 152.4930877685547 
	CER: 0.8506930470466614, accuracy: [0. 0.]
batch: 105, loss: 151.5292510986328 
	CER: 0.8060758709907532, accuracy: [0. 0.]
batch: 106, loss: 118.88917541503906 
	CER: 0.8801305890083313, accuracy: [0. 0.]
batch: 107, loss: 138.8520965576172 
	CER: 0.8922644853591919, accuracy: [0. 0.]
batch: 108, loss: 149.56491088867188 
	CER: 0.8245750665664673, accuracy: [0. 0.]
batch: 109, loss: 144.70059204101562 
	CER: 0.8107072114944458, accuracy: [0. 0.]
batch: 110, loss: 149.85879516601562 
	CER: 0.841476559638977, accuracy: [0. 0.]
batch: 111, loss: 125.43534851074219 
	CER: 0.8472437858581543, accuracy: [0. 0.]
batch: 112, loss: 139.1779327392578 
	CER: 0.8172774314880371, accuracy: [0. 0.]
batch: 113, loss: 166.6404571533203 
	CER: 0.82057124376297, accuracy: [0. 0.]
batch: 114, loss: 150.811

batch: 204, loss: 132.8701171875 
	CER: 0.8333649635314941, accuracy: [0. 0.]
batch: 205, loss: 143.64382934570312 
	CER: 0.8462909460067749, accuracy: [0. 0.]
batch: 206, loss: 164.98724365234375 
	CER: 0.8222321271896362, accuracy: [0. 0.]
batch: 207, loss: 120.80146789550781 
	CER: 0.8632028698921204, accuracy: [0. 0.]
batch: 208, loss: 154.42367553710938 
	CER: 0.8323196172714233, accuracy: [0. 0.]
batch: 209, loss: 163.6024169921875 
	CER: 0.8366256952285767, accuracy: [0. 0.]
batch: 210, loss: 147.52874755859375 
	CER: 0.8300259709358215, accuracy: [0. 0.]
batch: 211, loss: 164.29393005371094 
	CER: 0.8388442993164062, accuracy: [0. 0.]
batch: 212, loss: 143.3529052734375 
	CER: 0.8376094698905945, accuracy: [0. 0.]
batch: 213, loss: 154.1112823486328 
	CER: 0.8354901075363159, accuracy: [0. 0.]
batch: 214, loss: 157.00039672851562 
	CER: 0.8933976888656616, accuracy: [0. 0.]
batch: 215, loss: 152.4774169921875 
	CER: 0.8265345692634583, accuracy: [0. 0.]
batch: 216, loss: 192.36

batch: 306, loss: 129.1213836669922 
	CER: 0.8489353060722351, accuracy: [0. 0.]
batch: 307, loss: 142.7516326904297 
	CER: 0.8518063426017761, accuracy: [0. 0.]
batch: 308, loss: 156.36831665039062 
	CER: 0.8433851003646851, accuracy: [0. 0.]
batch: 309, loss: 142.96731567382812 
	CER: 0.8223608732223511, accuracy: [0. 0.]
batch: 310, loss: 139.2703094482422 
	CER: 0.843193531036377, accuracy: [0. 0.]
batch: 311, loss: 134.7535400390625 
	CER: 0.8493397831916809, accuracy: [0. 0.]
batch: 312, loss: 130.99375915527344 
	CER: 0.840259850025177, accuracy: [0. 0.]
batch: 313, loss: 149.4011993408203 
	CER: 0.8219806551933289, accuracy: [0. 0.]
batch: 314, loss: 172.36605834960938 
	CER: 0.8591233491897583, accuracy: [0. 0.]
batch: 315, loss: 138.76522827148438 
	CER: 0.836342990398407, accuracy: [0. 0.]
batch: 316, loss: 148.29086303710938 
	CER: 0.8267528414726257, accuracy: [0. 0.]
batch: 317, loss: 149.38400268554688 
	CER: 0.8383760452270508, accuracy: [0. 0.]
batch: 318, loss: 141.56

batch: 407, loss: 139.6058807373047 
	CER: 0.8248847126960754, accuracy: [0. 0.]
batch: 408, loss: 174.4819793701172 
	CER: 0.8486244082450867, accuracy: [0. 0.]
Error at  409
batch: 410, loss: 160.0550537109375 
	CER: 0.8072487115859985, accuracy: [0. 0.]
batch: 411, loss: 133.5679168701172 
	CER: 0.8581441640853882, accuracy: [0. 0.]
batch: 412, loss: 113.45585632324219 
	CER: 0.8756403923034668, accuracy: [0. 0.]
batch: 413, loss: 146.26333618164062 
	CER: 0.841185450553894, accuracy: [0. 0.]
batch: 414, loss: 159.85279846191406 
	CER: 0.8636702299118042, accuracy: [0. 0.]
batch: 415, loss: 140.33399963378906 
	CER: 0.8437564969062805, accuracy: [0. 0.]
batch: 416, loss: 125.41180419921875 
	CER: 0.8569666147232056, accuracy: [0. 0.]
batch: 417, loss: 158.0872344970703 
	CER: 0.834919273853302, accuracy: [0. 0.]
batch: 418, loss: 133.05735778808594 
	CER: 0.8684033155441284, accuracy: [0. 0.]
batch: 419, loss: 141.95547485351562 
	CER: 0.8502057790756226, accuracy: [0. 0.]
batch: 42

...

batch: 494, loss: 143.825927734375 
	CER: 0.8666019439697266, accuracy: [0. 0.]
batch: 495, loss: 143.42031860351562 
	CER: 0.8109771013259888, accuracy: [0. 0.]
batch: 496, loss: 187.7670440673828 
	CER: 0.8575231432914734, accuracy: [0. 0.]
batch: 497, loss: 122.96839904785156 
	CER: 0.8473766446113586, accuracy: [0. 0.]
batch: 498, loss: 160.61813354492188 
	CER: 0.868389904499054, accuracy: [0. 0.]
batch: 499, loss: 138.8515625 
	CER: 0.8395187258720398, accuracy: [0. 0.]
batch: 500, loss: 164.75424194335938 
	CER: 0.8625454902648926, accuracy: [0. 0.]
batch: 501, loss: 109.87969970703125 
	CER: 0.8749673962593079, accuracy: [0. 0.]
batch: 502, loss: 145.52481079101562 
	CER: 0.850997805595398, accuracy: [0. 0.]
batch: 503, loss: 141.82528686523438 
	CER: 0.8390388488769531, accuracy: [0. 0.]
batch: 504, loss: 143.39761352539062 
	CER: 0.8212321400642395, accuracy: [0. 0.]
batch: 505, loss: 128.02537536621094 
	CER: 0.8807676434516907, accuracy: [0. 0.]
batch: 506, loss: 150.497573

batch: 596, loss: 101.82525634765625 
	CER: 0.8971471786499023, accuracy: [0. 0.]
batch: 597, loss: 117.504150390625 
	CER: 0.8748137354850769, accuracy: [0. 0.]
batch: 598, loss: 139.05783081054688 
	CER: 0.8777510523796082, accuracy: [0. 0.]
batch: 599, loss: 157.77468872070312 
	CER: 0.8458712697029114, accuracy: [0. 0.]
batch: 600, loss: 169.41285705566406 
	CER: 0.8322746157646179, accuracy: [0. 0.]
batch: 601, loss: 141.20655822753906 
	CER: 0.8547555804252625, accuracy: [0. 0.]
batch: 602, loss: 133.2348175048828 
	CER: 0.8532710075378418, accuracy: [0. 0.]
batch: 603, loss: 112.654052734375 
	CER: 0.8905466198921204, accuracy: [0. 0.]
batch: 604, loss: 131.59988403320312 
	CER: 0.864879310131073, accuracy: [0. 0.]
batch: 605, loss: 150.02102661132812 
	CER: 0.8177503347396851, accuracy: [0. 0.]
batch: 606, loss: 112.83711242675781 
	CER: 0.8820298910140991, accuracy: [0. 0.]
batch: 607, loss: 141.97483825683594 
	CER: 0.8714786767959595, accuracy: [0. 0.]
batch: 608, loss: 120.

batch: 697, loss: 147.86459350585938 
	CER: 0.8329340219497681, accuracy: [0. 0.]
batch: 698, loss: 132.48886108398438 
	CER: 0.8578432202339172, accuracy: [0. 0.]
batch: 699, loss: 158.3120880126953 
	CER: 0.8122984766960144, accuracy: [0. 0.]
batch: 700, loss: 96.94963073730469 
	CER: 0.8237797021865845, accuracy: [0. 0.]
batch: 701, loss: 147.29379272460938 
	CER: 0.8280473947525024, accuracy: [0. 0.]
batch: 702, loss: 160.90090942382812 
	CER: 0.8481658697128296, accuracy: [0. 0.]
batch: 703, loss: 124.8587646484375 
	CER: 0.8493068814277649, accuracy: [0. 0.]
batch: 704, loss: 87.0694580078125 
	CER: 0.8991739153862, accuracy: [0. 0.]
batch: 705, loss: 137.97247314453125 
	CER: 0.8214108347892761, accuracy: [0. 0.]
batch: 706, loss: 118.4654541015625 
	CER: 0.8559412956237793, accuracy: [0. 0.]
batch: 707, loss: 146.80807495117188 
	CER: 0.8596604466438293, accuracy: [0. 0.]
batch: 708, loss: 164.7498779296875 
	CER: 0.7985983490943909, accuracy: [0. 0.]
batch: 709, loss: 147.2450

batch: 799, loss: 110.98292541503906 
	CER: 0.8535555601119995, accuracy: [0. 0.]
batch: 800, loss: 152.79624938964844 
	CER: 0.8495925664901733, accuracy: [0. 0.]
batch: 801, loss: 148.35775756835938 
	CER: 0.8648015856742859, accuracy: [0. 0.]
batch: 802, loss: 156.13690185546875 
	CER: 0.834196925163269, accuracy: [0. 0.]
batch: 803, loss: 170.3763427734375 
	CER: 0.8199930191040039, accuracy: [0. 0.]
batch: 804, loss: 111.24275970458984 
	CER: 0.8462473154067993, accuracy: [0. 0.]
batch: 805, loss: 146.9307098388672 
	CER: 0.8623937368392944, accuracy: [0. 0.]
batch: 806, loss: 143.98202514648438 
	CER: 0.8545783162117004, accuracy: [0. 0.]
batch: 807, loss: 181.18724060058594 
	CER: 0.8180135488510132, accuracy: [0. 0.]
batch: 808, loss: 164.8992462158203 
	CER: 0.8234437704086304, accuracy: [0. 0.]
batch: 809, loss: 147.41517639160156 
	CER: 0.8434950709342957, accuracy: [0. 0.]
batch: 810, loss: 129.80514526367188 
	CER: 0.8695511817932129, accuracy: [0. 0.]
batch: 811, loss: 16

batch: 82, loss: 148.1095428466797 
	CER: 0.8463028073310852, accuracy: [0. 0.]
batch: 83, loss: 103.96846771240234 
	CER: 0.8977304697036743, accuracy: [0. 0.]
batch: 84, loss: 154.8658447265625 
	CER: 0.8466815948486328, accuracy: [0. 0.]
batch: 85, loss: 118.1756820678711 
	CER: 0.8109983205795288, accuracy: [0. 0.]
batch: 86, loss: 130.32830810546875 
	CER: 0.8504695892333984, accuracy: [0. 0.]
batch: 87, loss: 155.58407592773438 
	CER: 0.831683337688446, accuracy: [0. 0.]
batch: 88, loss: 154.14859008789062 
	CER: 0.8345020413398743, accuracy: [0. 0.]
batch: 89, loss: 118.57550048828125 
	CER: 0.8786317706108093, accuracy: [0. 0.]
batch: 90, loss: 95.55057525634766 
	CER: 0.8947269320487976, accuracy: [0. 0.]
batch: 91, loss: 164.56417846679688 
	CER: 0.8536968231201172, accuracy: [0. 0.]
batch: 92, loss: 138.6974334716797 
	CER: 0.8454516530036926, accuracy: [0. 0.]
batch: 93, loss: 129.43023681640625 
	CER: 0.8540710210800171, accuracy: [0. 0.]
batch: 94, loss: 167.8454895019531

# Try getting actual predictions

In [ ]:
saver = tf.train.Saver()

with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.local_variables_initializer())
    sess.run(tf.tables_initializer())
    sess.run(tf.global_variables_initializer())
    
    # writer = tf.summary.FileWriter('./my_graph/LogRegNormal', sess.graph)
    n_batches = int(datasize / train_batch_size)
    for i in range(n_epochs):
        print("Starting epoch", i)
        sess.run(iterator.initializer)
        for j in range(n_batches):
            input_tensor_b, labels_b = sess.run(next_batch)
            _, cer, acc, loss, preddict, scp, pc = sess.run([train_op, CER, accuracy, loss_ctc, predictions_dict,
                                                          sparse_code_pred, pred_chars],
                         feed_dict={input_tensor: input_tensor_b, labels: labels_b})
            
            pickle.dump([cer, acc, loss, preddict, scp, pc], open(output_model_dir+"/preds2.pkl", "wb"))
            print('batch: {0}, CER: {1}, accuracy: {2}, loss: {3}'.format(j, cer, acc, loss))
            if j > 0: break # for now don't do all of them

    print('Total time: {0} seconds'.format(time.time() - start_time))

In [ ]:
for b in preddict["words"]:
    print(len(b), end="\t")

In [ ]:
print()
for i in range(len(preddict["prob"][0][3])):
    print(i, alphabet[i], preddict["prob"][0][3][i])